# Aggregating and Visualizing LiDAR Data from NuScenes
#### Step 1: Initialize the Scene
- Identify the target scene by its name (e.g., `scene-0103`) in the NuScenes dataset.
- Retrieve the first sample token from the scene to start processing frames sequentially.

**Reasoning**: To begin the aggregation process, the target scene must first be identified by name. The `first_sample_token` is then used to retrieve the starting point for sequential processing of frames in the scene. This ensures a systematic and complete traversal of all frames.

#### Step 2: Iterate Through Frames
- Use the `sample['next']` token to move through all frames in the scene.
- For each frame:
  - Retrieve the LiDAR data token (`LIDAR_TOP`).
  - Load the raw point cloud using `LidarPointCloud.from_file()`.

**Reasoning**: Iterating through the frames allows for the sequential processing of LiDAR data, ensuring that all data along the vehicle's trajectory is captured. By accessing the `sample['next']` token, the algorithm moves from one frame to the next until the entire scene is covered. This systematic approach ensures no data is missed.

#### Step 3: Transform Points to Global Coordinates
- Retrieve the sensor calibration data to compute the transformation from the sensor's local frame to the ego (vehicle) frame.
  - Combine the sensor's translation and rotation into a transformation matrix.
- Retrieve the ego pose data to compute the transformation from the ego frame to the global coordinate frame.
  - Combine the ego vehicle's translation and rotation into another transformation matrix.
- Multiply the two matrices to derive the transformation from the sensor frame to the global frame.
- Apply this transformation to the LiDAR points using `.transform()`.

**Reasoning**: LiDAR points are initially in the sensor's local coordinate system and need to be transformed into the global coordinate frame for proper alignment across frames. This is achieved through two transformations:
- **Sensor to Ego Frame**: Using the sensor's calibration data, the points are transformed relative to the vehicle's coordinate system.
- **Ego to Global Frame**: Using the vehicle's ego pose data, the points are further transformed into the global coordinate frame.
Combining these transformations ensures that all points are accurately aligned in a common global frame.

#### Step 4: Aggregate Transformed Points
- Convert the transformed points into an Open3D `PointCloud` object.
- Add the points to an aggregated point cloud initialized at the start.
- Repeat this process for all frames until the end of the scene is reached.

**Reasoning**: After transforming the points into the global frame, they are added to an aggregated point cloud. Using an Open3D `PointCloud` object simplifies the management and visualization of these points. Iterating through all frames and aggregating their points creates a comprehensive representation of the scene.


#### Step 5: Save and Visualize
- Save the aggregated point cloud as a `.pcd` file using `o3d.io.write_point_cloud()`.
- Visualize the aggregated point cloud using Open3D’s visualization tools.

**Reasoning**: Saving the aggregated point cloud as a `.pcd` file allows for further analysis and reuse of the data. Visualizing the point cloud using Open3D provides immediate feedback on the quality and completeness of the aggregation process, enabling verification of the results.

In [1]:
import numpy as np
import open3d as o3d
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix, Quaternion


def transform_lidar_to_global(nusc, lidar_token):
    """
    Transform LiDAR point cloud data from sensor coordinates to global coordinates.

    Args:
        nusc (NuScenes): Instance of the NuScenes dataset.
        lidar_token (str): Token for the LiDAR data.

    Returns:
        np.ndarray: Transformed point cloud as a (N, 3) array.
    """
    # Get LiDAR metadata
    lidar_data = nusc.get("sample_data", lidar_token)
    lidar_path = nusc.get_sample_data_path(lidar_token)

    # Load LiDAR points
    lidar_points = LidarPointCloud.from_file(lidar_path)

    # Get ego pose and calibration data
    ego_pose = nusc.get("ego_pose", lidar_data["ego_pose_token"])
    calibration = nusc.get("calibrated_sensor", lidar_data["calibrated_sensor_token"])

    # Compute transformations
    sensor_to_ego = transform_matrix(
        calibration["translation"], Quaternion(calibration["rotation"]), inverse=False
    )
    ego_to_global = transform_matrix(
        ego_pose["translation"], Quaternion(ego_pose["rotation"]), inverse=False
    )
    sensor_to_global = ego_to_global @ sensor_to_ego

    # Apply the global transformation
    lidar_points.transform(sensor_to_global)

    return lidar_points.points.T[:, :3]  # Return (N, 3) point cloud


def aggregate_scene_points(nusc, scene_name):
    """
    Aggregate all LiDAR frames in a scene into a single global point cloud.

    Args:
        nusc (NuScenes): Instance of the NuScenes dataset.
        scene_name (str): Name of the target scene to process.

    Returns:
        o3d.geometry.PointCloud: Aggregated global point cloud.
    """
    # Find the scene and initialize the global cloud
    scene_info = next(scene for scene in nusc.scene if scene["name"] == scene_name)
    sample_token = scene_info["first_sample_token"]
    global_cloud = o3d.geometry.PointCloud()

    while sample_token:
        # Access the sample and get the LiDAR token
        sample = nusc.get("sample", sample_token)
        lidar_token = sample["data"]["LIDAR_TOP"]

        # Transform LiDAR points to the global frame
        lidar_points = transform_lidar_to_global(nusc, lidar_token)

        # Convert to Open3D point cloud and add to the global cloud
        lidar_cloud = o3d.geometry.PointCloud()
        lidar_cloud.points = o3d.utility.Vector3dVector(lidar_points)
        global_cloud += lidar_cloud

        # Move to the next sample
        sample_token = sample["next"] if sample["next"] else None

    return global_cloud


def visualize_point_cloud(point_cloud, window_title="Point Cloud Visualization"):
    """
    Visualize a point cloud using Open3D.

    Args:
        point_cloud (o3d.geometry.PointCloud): Point cloud to visualize.
        window_title (str): Title for the visualization window.
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=window_title)

    # Configure rendering settings
    render_option = vis.get_render_option()
    render_option.point_size = 1.0

    vis.add_geometry(point_cloud)
    vis.run()
    vis.destroy_window()


if __name__ == "__main__":
    # Initialize NuScenes dataset
    nusc = NuScenes(version="v1.0-mini", dataroot="Dataset/v1.0-mini", verbose=True)

    # Aggregate point clouds for a given scene
    target_scene = "scene-0103"  # Specify the target scene
    aggregated_cloud = aggregate_scene_points(nusc, target_scene)

    # Save the aggregated point cloud
    output_file = "step1_aggregated_cloud.pcd"
    print(f"Saving aggregated point cloud with {len(aggregated_cloud.points)} points to {output_file}")
    o3d.io.write_point_cloud(output_file, aggregated_cloud)

    # Visualize the point cloud
    visualize_point_cloud(aggregated_cloud, window_title="Aggregated Point Cloud")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.427 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
Saving aggregated point cloud with 1388928 points to step1_aggregated_cloud.pcd


# Filtering Moving and Static Objects in Aggregated Point Cloud

#### Step 1: Compute Object Velocity
- For each annotation, calculate the velocity of the object using its current and previous positions.
  - Retrieve the current and previous translations from the annotation metadata.
  - Calculate the displacement between the two positions.
  - Compute the velocity by dividing the displacement by the time difference between the two frames.
  - Return zero velocity if there is no previous annotation.

**Reasoning**: Velocity is a critical factor in distinguishing moving objects from static ones. By comparing the current and previous positions of an object, we can calculate its displacement over time. Dividing the displacement by the time interval gives the velocity, which can be used to classify objects as moving or static. If no previous data is available, a velocity of zero is assigned to avoid false classification.


#### Step 2: Filter Moving and Static Points
- Initialize boolean masks for moving and static points in the point cloud.
  - `moving_mask`: Marks points corresponding to moving objects.
  - `static_mask`: Marks points corresponding to static objects.
  - `ground_mask`: Identifies ground points based on a height threshold.

- For each annotation:
  - Retrieve the velocity of the object.
  - Create a bounding box for the object using its translation, size, and rotation.
  - Identify points within the bounding box and exclude ground points.
  - Mark points as moving if the object's velocity exceeds the velocity threshold.
  - Mark points as static otherwise.

**Reasoning**: Points belonging to moving objects need to be identified and separated from the static environment. Boolean masks (`moving_mask`, `static_mask`, and `ground_mask`) are used to filter points based on their association with moving objects, static objects, or the ground plane. The ground mask ensures that low-lying ground points are excluded from object filtering. By using the bounding box of each object and comparing it to the aggregated points, we can accurately mark points as moving or static based on the velocity threshold.

#### Step 3: Separate Moving and Static Clouds
- Extract points from the aggregated cloud based on the masks:
  - `moving_points`: Points corresponding to moving objects.
  - `static_points`: Points corresponding to static objects.

- Convert these points into Open3D `PointCloud` objects and assign distinct colors for visualization:
  - Moving points: Red.
  - Static points: Cyan.

**Reasoning**: After identifying moving and static points, separating them into distinct clouds is essential for analysis and visualization. Open3D `PointCloud` objects are used to store and visualize these points. Assigning distinct colors (e.g., red for moving and cyan for static) enhances the visual distinction between moving and static elements in the scene, aiding interpretation.


#### Step 4: Save and Visualize the Results
- Save the static and moving point clouds as `.pcd` files.
- Combine the two clouds and visualize them interactively using Open3D.

**Reasoning**: Saving the static and moving point clouds as `.pcd` files ensures the results can be reused or analyzed in the future. Visualization provides immediate feedback, enabling verification of the filtering process and highlighting the separation between moving and static components. Combining the clouds and using an interactive viewer allows for a comprehensive overview of the scene.


In [2]:
import numpy as np
import open3d as o3d
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, Box
from nuscenes.utils.geometry_utils import transform_matrix, points_in_box, Quaternion


def compute_velocity(annotation, nusc):
    """
    Calculate the velocity of an object from its annotation data.
    """
    if annotation.get('prev'):
        try:
            # Calculate velocity using previous annotation
            current_translation = np.array(annotation['translation'])
            current_sample = nusc.get('sample', annotation['sample_token'])
            current_timestamp = current_sample['timestamp']

            prev_ann = nusc.get('sample_annotation', annotation['prev'])
            prev_translation = np.array(prev_ann['translation'])
            prev_sample = nusc.get('sample', prev_ann['sample_token'])
            prev_timestamp = prev_sample['timestamp']

            displacement = current_translation - prev_translation
            time_delta = (current_timestamp - prev_timestamp) / 1e6  # Convert from microseconds to seconds
            return np.linalg.norm(displacement) / time_delta
        except KeyError as e:
            print(f"KeyError in compute_velocity: {e}")
            return 0.0
    return 0.0  # Return zero if no previous annotation exists


def filter_moving_objects(agg_pc, annotations, nusc, velocity_threshold=0.25, ground_threshold=0.3):
    """
    Identify points corresponding to moving objects in the aggregated point cloud.

    Parameters:
        agg_pc (np.ndarray): Aggregated global point cloud (Nx3).
        annotations (list): List of annotations for the scene.
        nusc (NuScenes): NuScenes instance for accessing metadata.
        velocity_threshold (float): Minimum velocity to consider an object as moving.
        ground_threshold (float): Height threshold to filter ground points.

    Returns:
        o3d.geometry.PointCloud: Static point cloud.
        o3d.geometry.PointCloud: Moving point cloud.
    """
    # Initialize masks
    moving_mask = np.zeros(agg_pc.shape[0], dtype=bool)  # Initialize all as non-moving
    static_mask = np.ones(agg_pc.shape[0], dtype=bool)   # Initialize all as static

    # Ground points mask based on height threshold
    ground_mask = agg_pc[:, 2] < ground_threshold  # Identify ground points (z < ground_threshold)

    for ann in annotations:
        if not ('translation' in ann and 'size' in ann and 'rotation' in ann):
            print(f"Skipping invalid annotation: {ann}")
            continue

        velocity = compute_velocity(ann, nusc)

        # Bounding box for the object
        box = Box(
            ann['translation'],
            ann['size'],
            Quaternion(ann['rotation'])
        )

        # Get points inside the bounding box
        points_in_bbox = points_in_box(box, agg_pc.T)

        # Exclude ground points from consideration
        points_in_bbox = points_in_bbox & ~ground_mask

        if velocity > velocity_threshold:
            moving_mask[points_in_bbox] = True
        else:
            static_mask[points_in_bbox] = False

    # Extract static and moving points
    moving_points = agg_pc[moving_mask]
    static_points = agg_pc[static_mask]

    # Create Open3D point clouds
    static_cloud = o3d.geometry.PointCloud()
    static_cloud.points = o3d.utility.Vector3dVector(static_points)
    static_cloud.paint_uniform_color([0.5, 0.9, 1]) 

    moving_cloud = o3d.geometry.PointCloud()
    moving_cloud.points = o3d.utility.Vector3dVector(moving_points)
    moving_cloud.paint_uniform_color([1, 0, 0])  # Red for moving

    return static_cloud, moving_cloud


if __name__ == "__main__":
    # Initialize NuScenes
    nusc = NuScenes(version='v1.0-mini', dataroot='Dataset/v1.0-mini', verbose=True)

    # Load the aggregated point cloud
    aggregated_cloud = o3d.io.read_point_cloud("step1_aggregated_cloud.pcd")
    agg_points = np.asarray(aggregated_cloud.points)

    # Extract scene annotations
    scene_name = "scene-0103"
    scene = next(scene for scene in nusc.scene if scene['name'] == scene_name)
    sample_token = scene['first_sample_token']
    annotations = []

    while sample_token:
        sample = nusc.get('sample', sample_token)
        for token in sample['anns']:
            annotations.append(nusc.get('sample_annotation', token))
        sample_token = sample['next'] if sample['next'] else None

    # Filter moving and static objects
    velocity_threshold = 0.25  # Velocity threshold for moving objects
    ground_threshold = 0.3  # Height threshold for ground points
    static_cloud, moving_cloud = filter_moving_objects(agg_points, annotations, nusc, velocity_threshold, ground_threshold)

    # Save point clouds
    o3d.io.write_point_cloud("static.pcd", static_cloud)
    o3d.io.write_point_cloud("moving.pcd", moving_cloud)

    # Visualize combined cloud
    combined_cloud = static_cloud + moving_cloud
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Aggregated Point Cloud with Moving Objects")
    render_option = vis.get_render_option()
    render_option.point_size = 0.6
    vis.add_geometry(combined_cloud)
    vis.run()
    vis.destroy_window()


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.382 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


# Enriching Static Cloud with Camera Colors in NuScenes

#### Step 1: Load Camera Data
- Retrieve camera data for the specified sample token and camera channel.
  - Extract camera calibration, ego pose, and the image path from NuScenes metadata.

**Reason**: To project points onto the image plane, we need camera-specific information. By extracting the calibration data (intrinsic and extrinsic), ego pose, and image path from NuScenes metadata, we can accurately map points from the global frame to the camera frame and associate them with the corresponding image pixels.

#### Step 2: Transform Points to Camera Frame
- Convert 3D points from the global coordinate frame to the camera frame.
  - Compute the transformation matrix by combining camera pose and calibration data.
  - Apply the transformation to the 3D points.

**Reason**: 3D points exist in the global coordinate system, but the camera captures images in its local coordinate system. By applying the transformation matrices for the ego pose and the camera calibration, we can reposition the points relative to the camera's viewpoint, enabling correct projection onto the image plane.
#### Step 3: Project Points to Image Plane
- Project the 3D points in the camera frame to 2D pixel coordinates on the image plane.
  - Use the intrinsic matrix of the camera for projection.
  - Normalize by the depth of the points to get pixel coordinates.

**Reason**: Cameras capture 3D scenes as 2D images. Using the intrinsic matrix of the camera, we project the transformed 3D points into 2D pixel coordinates. Normalizing by the depth ensures the projection accurately represents the points' locations on the image plane.

#### Step 4: Enrich Points with Colors
- For each valid 3D point:
  - Verify it is within the camera's field of view.
  - Extract its RGB value from the corresponding pixel in the image.
  - Accumulate colors for points visible in multiple camera views.

**Reason**: To create a visually enriched point cloud, each 3D point must be assigned an RGB color. By checking which points lie within the camera's field of view and depth range, we can associate valid points with pixel colors from the corresponding camera image. Accumulating colors from multiple camera views ensures better coverage and realistic representation.

#### Step 5: Process Scene
- Iterate through all samples in the scene and process the static cloud.
  - For each sample, retrieve the associated camera images.
  - Enrich the static cloud with RGB colors from these images.

**Reason**: Scenes in NuScenes consist of multiple frames. Iterating through all samples ensures comprehensive processing of the static cloud, enriching it with colors from all relevant camera images. This step provides a holistic view of the scene's environment.

#### Step 6: Combine and Visualize
- Combine the enriched static cloud and the moving cloud.
- Save the resulting point cloud as a `.pcd` file.
- Visualize the enhanced point cloud interactively.

**Reason**: Combining the enriched static cloud with the moving cloud creates a complete representation of the scene. Saving the combined point cloud allows for future analysis, and visualizing it provides immediate feedback on the quality of enrichment and alignment.

In [ ]:
import numpy as np
from PIL import Image
import open3d as o3d
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import transform_matrix, Quaternion

def load_camera_data(nusc, sample_token, camera_channel):
    """
    Retrieve camera data, including calibration, ego pose, and image path.
    """
    sample = nusc.get("sample", sample_token)
    cam_data = nusc.get("sample_data", sample["data"][camera_channel])
    cam_calib = nusc.get("calibrated_sensor", cam_data["calibrated_sensor_token"])
    cam_pose = nusc.get("ego_pose", cam_data["ego_pose_token"])
    img_path = nusc.get_sample_data_path(cam_data["token"])
    return cam_calib, cam_pose, img_path

def transform_to_camera_frame(points, cam_calib, cam_pose):
    """
    Transform 3D points from the global frame to the camera frame.
    """
    world_to_camera = np.linalg.inv(
        transform_matrix(cam_pose["translation"], Quaternion(cam_pose["rotation"]), inverse=False) @
        transform_matrix(cam_calib["translation"], Quaternion(cam_calib["rotation"]), inverse=False)
    )
    points_homogeneous = np.hstack((points, np.ones((points.shape[0], 1))))
    return (world_to_camera @ points_homogeneous.T).T

def project_to_image(points_camera, intrinsic_matrix):
    """
    Project 3D points in the camera frame to 2D pixel coordinates in the image.
    """
    depth = points_camera[:, 2]
    points_2d = (intrinsic_matrix @ points_camera[:, :3].T).T
    points_2d[:, :2] /= depth[:, None]
    return points_2d, depth

def enrich_with_camera_colors(static_cloud, nusc, sample_token, camera_channels):
    """
    Enrich the static cloud with RGB colors from multiple camera images.
    """
    points = np.asarray(static_cloud.points)
    colors = np.zeros((points.shape[0], 3))
    visibility = np.zeros(points.shape[0])

    for channel in camera_channels:
        cam_calib, cam_pose, img_path = load_camera_data(nusc, sample_token, channel)
        intrinsic = np.array(cam_calib["camera_intrinsic"])
        image = np.asarray(Image.open(img_path))

        # Transform and project points
        points_camera = transform_to_camera_frame(points, cam_calib, cam_pose)
        points_2d, depth = project_to_image(points_camera, intrinsic)

        # Filter valid points
        valid_mask = (
            (depth > 0) &
            (points_2d[:, 0] >= 0) & (points_2d[:, 0] < image.shape[1]) &
            (points_2d[:, 1] >= 0) & (points_2d[:, 1] < image.shape[0])
        )

        # Assign colors
        valid_points_2d = points_2d[valid_mask, :2].astype(int)
        if valid_mask.sum() > 0:
            colors[valid_mask] += image[valid_points_2d[:, 1], valid_points_2d[:, 0]] / 255.0
            visibility[valid_mask] += 1

  
    static_cloud.colors = o3d.utility.Vector3dVector(colors / visibility[:, None])
    return static_cloud

def process_scene(nusc, scene_name, static_file, moving_file, output_file):
    """
    Process a scene by enriching static and moving clouds and combining them.
    """
    static_cloud = o3d.io.read_point_cloud(static_file)
    moving_cloud = o3d.io.read_point_cloud(moving_file)
    moving_cloud.colors = o3d.utility.Vector3dVector(np.full((len(moving_cloud.points), 3), 0.5))  # Grayscale

    # Process static cloud with camera images
    scene = next(s for s in nusc.scene if s["name"] == scene_name)
    sample_token = scene["first_sample_token"]

    while sample_token:
        sample = nusc.get("sample", sample_token)
        camera_channels = [key for key in sample["data"].keys() if "CAM" in key]
        static_cloud = enrich_with_camera_colors(static_cloud, nusc, sample_token, camera_channels)
        sample_token = sample["next"]  # Move to the next sample

    # Combine and save the clouds
    combined_cloud = static_cloud + moving_cloud
    o3d.io.write_point_cloud(output_file, combined_cloud)

    # Visualize the combined point cloud
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Enhanced Point Cloud")
    render_options = vis.get_render_option()
    render_options.background_color = np.array([0, 0, 0])
    render_options.point_size = 1.0
    vis.add_geometry(combined_cloud)
    vis.run()
    vis.destroy_window()

if __name__ == "__main__":
    nusc = NuScenes(version="v1.0-mini", dataroot="Dataset/v1.0-mini", verbose=True)
    process_scene(
        nusc,
        scene_name="scene-0103",
        static_file="static.pcd",
        moving_file="moving.pcd",
        output_file="enhanced_cloud.pcd"
    )


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.378 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


/tmp/ipykernel_22252/263384370.py:69: RuntimeWarning: invalid value encountered in divide
  static_cloud.colors = o3d.utility.Vector3dVector(colors / visibility[:, None])
